# PVICE & RELOG - MassFlows Calculations

## 1. Initial setup

### Load libraries

In [1]:
import PV_ICE
import numpy as np
import pandas as pd
import os,sys
import matplotlib.pyplot as plt

### Select the baselines and simulation folders

In [2]:
cwd = os.getcwd()
baselinefolder = os.path.join(cwd, 'baselines')
testfolder = os.path.join(cwd, 'TEMP')


In [3]:
SFscenarios = ['95-by-35_Elec.Adv_DR_cSi', '95-by-35_Elec.Adv_DR_CdTe']

### Reading GIS inputs

In [4]:
from geopy.geocoders import Nominatim
from geopy.point import Point
# initialize Nominatim API
geolocator = Nominatim(user_agent="geoapiExercises")

In [5]:
GISfile = os.path.join(baselinefolder, 'gis_centroid_n.xlsx')
GIS = pd.read_excel(GISfile)
GIS = GIS.set_index('id')

## 2. Load PCA baselines, create the 2 Scenarios and assign baselines

Keeping track of each scenario as its own PV ICE Object.

Select the method folder you want to run (uncomment your choice). There are three choices:
1. Method 1: Uses the raw regionalized capacity by ReEEDS, this creates a very uneven peak of wastes.
2. Method 2: Uses ordered wastes between 2021 to 2035 and 2046 to 2050. Still creates unrealistic peaks.
3. Method 3: Uses the cummulative capacity between 2021 to 2035 and 2034 to 2050 to create a logarithmic growth of waste (this method is being tested, not validated yet, and subjected to ongoing changes).

In [6]:
# method = 'Method1' # Raw waste creation with the weird ReEDS 
# method = 'Method2' # Ordered waste 
method = 'Method3' # Uses the cummulative capacity between 2021 to 2035 and 2034 to 2050 to create a logarithmic growth of waste (it is still on the works)

### Scenario creation

In [7]:
reedsFile = os.path.join(cwd, 'baselines','December Core Scenarios ReEDS Outputs Solar Futures v3a.xlsx')
print ("Input file is stored in %s" % reedsFile)
REEDSInput = pd.read_excel(reedsFile, sheet_name="new installs PV")
rawdf = REEDSInput.copy()
rawdf.drop(columns=['State'], inplace=True)
rawdf.drop(columns=['Tech'], inplace=True) #tech=pvtotal from "new installs PV sheet", so can drop
rawdf.set_index(['Scenario','Year','PCA'], inplace=True)
PCAs = list(rawdf.unstack(level=2).iloc[0].unstack(level=0).index.unique())

Input file is stored in /Users/mmendez/Documents/Postdoc/Software_dev/RICE/1_PV_ICE_waste_calculations/baselines/December Core Scenarios ReEDS Outputs Solar Futures v3a.xlsx


In [8]:
#for ii in range (0, 1): #len(scenarios):
i = 0
r1 = PV_ICE.Simulation(name=SFscenarios[i], path=testfolder)

for jj in range (0, len(PCAs)): 
    filetitle = SFscenarios[i]+'_'+PCAs[jj]+'.csv'
    filetitle = os.path.join(testfolder, f'PCAs_RELOG_{method}', filetitle)    
    r1.createScenario(name=PCAs[jj], massmodulefile=filetitle)
    r1.scenario[PCAs[jj]].addMaterials(['glass', 'silicon', 'silver', 'copper', 'aluminium_frames', 'encapsulant', 'backsheet'], baselinefolder=baselinefolder)
    # All -- but these where not included in the Reeds initial study as we didnt have encapsulant or backsheet
    # r1.scenario[PCAs[jj]].addMaterials(['glass', 'silicon', 'silver', 'copper', 'aluminium_frames', 'encapsulant', 'backsheet'], baselinefolder=r'..\baselines')
    r1.scenario[PCAs[jj]].latitude = GIS.loc[PCAs[jj]].lat
    r1.scenario[PCAs[jj]].longitude = GIS.loc[PCAs[jj]].long

r1.trim_Years(startYear=2010, endYear=2050)

i = 1
r2 = PV_ICE.Simulation(name=SFscenarios[i], path=testfolder)

for jj in range (0, len(PCAs)): 
    filetitle = SFscenarios[i]+'_'+PCAs[jj]+'.csv'
    filetitle = os.path.join(testfolder, f'PCAs_RELOG_{method}', filetitle)        
    r2.createScenario(name=PCAs[jj], massmodulefile=filetitle)
    # MAC Add here the materials you want.
    r2.scenario[PCAs[jj]].addMaterials(['cadmium', 'tellurium', 'glass_cdte', 'aluminium_frames_cdte', 'encapsulant_cdte', 'copper_cdte'], baselinefolder=baselinefolder)
    # All -- but these where not included in the Reeds initial study as we didnt have encapsulant or backsheet
    # r2.scenario[PCAs[jj]].addMaterials(['glass', 'silicon', 'silver', 'copper', 'aluminium_frames', 'encapsulant', 'backsheet'], baselinefolder=r'..\baselines')
    r2.scenario[PCAs[jj]].latitude = GIS.loc[PCAs[jj]].lat
    r2.scenario[PCAs[jj]].longitude = GIS.loc[PCAs[jj]].long

r2.trim_Years(startYear=2010, endYear=2050)


path = /Users/mmendez/Documents/Postdoc/Software_dev/RICE/1_PV_ICE_waste_calculations/TEMP
No energy data loaded. Skipping for all next scenarios and materials
Data trimmed for Mass, years now encompass  2010  to  2050
Data trimmed for Mass, years now encompass  2010  to  2050
Data trimmed for Mass, years now encompass  2010  to  2050
Data trimmed for Mass, years now encompass  2010  to  2050
Data trimmed for Mass, years now encompass  2010  to  2050
Data trimmed for Mass, years now encompass  2010  to  2050
Data trimmed for Mass, years now encompass  2010  to  2050
Data trimmed for Mass, years now encompass  2010  to  2050
Data trimmed for Mass, years now encompass  2010  to  2050
Data trimmed for Mass, years now encompass  2010  to  2050
Data trimmed for Mass, years now encompass  2010  to  2050
Data trimmed for Mass, years now encompass  2010  to  2050
Data trimmed for Mass, years now encompass  2010  to  2050
Data trimmed for Mass, years now encompass  2010  to  2050
Data trimmed f

### Set characteristics for Manufacturing (probably don't want to inflate this as the waste happens elsewhere, just want EOL


In [9]:
PERFECTMFG = True
# Set to false if I want to see how much goes to mnf waste
if PERFECTMFG:
    r1.scenMod_PerfectManufacturing()
    r2.scenMod_PerfectManufacturing()
    title_Method = 'PVICE_PerfectMFG'
else:
    title_Method = 'PVICE'

## 3. Calculate Mass Flow

In [10]:
r1.calculateMassFlow()
r2.calculateMassFlow()

>>>> Calculating Material Flows <<<<

Working on Scenario:  p1
********************
Finished Area+Power Generation Calculations
==> Working on Material :  glass
==> Working on Material :  silicon
==> Working on Material :  silver
==> Working on Material :  copper
==> Working on Material :  aluminium_frames
==> Working on Material :  encapsulant
==> Working on Material :  backsheet
Working on Scenario:  p10
********************
Finished Area+Power Generation Calculations
==> Working on Material :  glass
==> Working on Material :  silicon
==> Working on Material :  silver
==> Working on Material :  copper
==> Working on Material :  aluminium_frames
==> Working on Material :  encapsulant
==> Working on Material :  backsheet
Working on Scenario:  p100
********************
Finished Area+Power Generation Calculations
==> Working on Material :  glass
==> Working on Material :  silicon
==> Working on Material :  silver
==> Working on Material :  copper
==> Working on Material :  aluminium_fram

In [11]:
print("PCAs:", r1.scenario.keys())
print("Module Keys:", r1.scenario[PCAs[jj]].dataIn_m.keys())
print("Material Keys: ", r1.scenario[PCAs[jj]].material['glass'].matdataIn_m.keys())

PCAs: dict_keys(['p1', 'p10', 'p100', 'p101', 'p102', 'p103', 'p104', 'p105', 'p106', 'p107', 'p108', 'p109', 'p11', 'p110', 'p111', 'p112', 'p113', 'p114', 'p115', 'p116', 'p117', 'p118', 'p119', 'p12', 'p120', 'p121', 'p122', 'p123', 'p124', 'p125', 'p126', 'p127', 'p128', 'p129', 'p13', 'p130', 'p131', 'p132', 'p133', 'p134', 'p14', 'p15', 'p16', 'p17', 'p18', 'p19', 'p2', 'p20', 'p21', 'p22', 'p23', 'p24', 'p25', 'p26', 'p27', 'p28', 'p29', 'p3', 'p30', 'p31', 'p32', 'p33', 'p34', 'p35', 'p36', 'p37', 'p38', 'p39', 'p4', 'p40', 'p41', 'p42', 'p43', 'p44', 'p45', 'p46', 'p47', 'p48', 'p49', 'p5', 'p50', 'p51', 'p52', 'p53', 'p54', 'p55', 'p56', 'p57', 'p58', 'p59', 'p6', 'p60', 'p61', 'p62', 'p63', 'p64', 'p65', 'p66', 'p67', 'p68', 'p69', 'p7', 'p70', 'p71', 'p72', 'p73', 'p74', 'p75', 'p76', 'p77', 'p78', 'p79', 'p8', 'p80', 'p81', 'p82', 'p83', 'p84', 'p85', 'p86', 'p87', 'p88', 'p89', 'p9', 'p90', 'p91', 'p92', 'p93', 'p94', 'p95', 'p96', 'p97', 'p98', 'p99'])
Module Keys: Index

In [12]:
"""
r1.plotScenariosComparison(keyword='Cumulative_Area_disposedby_Failure')
r1.plotMaterialComparisonAcrossScenarios(material='silicon', keyword='mat_Total_Landfilled')
r1.scenario['p1'].dataIn_m.head(21)
r2.scenario['p1'].dataIn_m.head(21)
r3.scenario['p1'].dataIn_m.head(21)
"""
pass

## 4. Aggregate & Save Data

In [13]:
r1.aggregateResults()
r2.aggregateResults()

/Users/mmendez/Documents/Postdoc/Software_dev/PV_ICE/PV_ICE/main.py:1805: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  USyearly['newInstalledCapacity_'+self.name+'_'+scen+'_[MW]'] = self.scenario[scen].dataIn_m[keywd1]
/Users/mmendez/Documents/Postdoc/Software_dev/PV_ICE/PV_ICE/main.py:1805: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  USyearly['newInstalledCapacity_'+self.name+'_'+scen+'_[MW]'] = self.scenario[scen].dataIn_m[keywd1]
/Users/mmendez/Documents/Postdoc/Software_dev/PV_ICE/PV_ICE/main.py:1805: PerformanceWarning: 

(      VirginStock_cadmium_95-by-35_Elec.Adv_DR_CdTe_p1_[Tonnes]  \
 year                                                              
 2010                                           0.000133           
 2011                                           0.012354           
 2012                                           0.014132           
 2013                                           0.026922           
 2014                                           0.092491           
 2015                                           0.045304           
 2016                                           0.069158           
 2017                                           0.053016           
 2018                                           0.057374           
 2019                                           0.136830           
 2020                                           0.164196           
 2021                                           0.011232           
 2022                                           

In [14]:
datay = r1.USyearly
datac = r1.UScum


In [15]:
datay_CdTe = r2.USyearly
datac_CdTe = r2.UScum

In [16]:
filter_colc = [col for col in datay if col.startswith('WasteEOL')]
datay[filter_colc].to_csv('PVICE_RELOG_PCA_cSi_WasteEOL.csv')
filter_colc = [col for col in datay_CdTe if col.startswith('WasteEOL')]
datay_CdTe[filter_colc].to_csv('PVICE_RELOG_PCA_CdTe_WasteEOL.csv')